In [ ]:
!pip install --upgrade google-cloud-aiplatform

In [1]:
import requests
import vertexai
from vertexai.preview.language_models import TextGenerationModel
import json
import pandas as pd
from vertexai.preview.generative_models import GenerativeModel, Part
import base64
import vertexai.preview.generative_models as generative_models
import re, json

In [2]:
from google.cloud import storage

# Initialize the client
client = storage.Client(project="bigquerycourse-onetwothree")

# Define your bucket name
bucket_name = 'contracts_poc'

# Create a bucket object
bucket = client.bucket(bucket_name)

# List of URIs for PDFs
pdf_uris = []

# List all objects in the bucket
for blob in client.list_blobs(bucket_name):
  if blob.name.endswith('.pdf'):
    # Get the URI for the PDF object (gs://bucket_name/object_name)
    pdf_uri = f"gs://{bucket_name}/{blob.name}"
    pdf_uris.append(pdf_uri)
    print(f"Found PDF URI: {pdf_uri}")

Found PDF URI: gs://contracts_poc/Dunn Transportation.pdf
Found PDF URI: gs://contracts_poc/Life Quest.pdf


In [3]:
prompt = """
    Role
    ---------
    You are an expert at extracting key information from supplier contracts
    Output as JSON object

    Task
    ---------
    I will feed you a supplier contract and I want you to extract the following information from it
    Supplier, Purchaser, Description of Services, Length of Service, Break Clause Included, Description of Break Clause,
    Value of Contract, Currency, Signature Date, Contract Expiry Date, Payment Terms,
    Total Contract Lifetime Value (sum of All payments if contract runs for full term),
    Supplier Country, Purchase Country, Work Country, Contract Duration, Number of Pages,
    Payment Terms

    Example Output
    ------------
    {
      "Supplier": "Police Ordnance Company Inc",
      "Purchaser": "The City of Tempe",
      "Description_of_Services": "Purchase of less lethal ammunition for ARWEN launchers.",
      "Length_of_Service": "One year with the possibility of four additional one-year renewals.",
      "Break_Clause_Included": "Yes",
      "Description_of_Break_Clause": "The City can terminate the agreement with a 90-day written notice.",
      "Value_of_Contract": "Not to exceed $99,999 per year",
      "Currency": "USD",
      "Signature_Date": "June 29, 2020",
      "Contract_Expiry_Date": "June 29, 2025",
      "Payment Terms": "30 days",
      "Total_Contract_Lifetime_Value": "$1,000,000",
      "Supplier_Country": "United States",
      "Purchaser_Country": "United States",
      "Work_Country": "United States",
      "Contract_Duration": "10 years",
      "Number_of_Pages": "10 pages",
      "Payment Terms": "30 days after invoice"
    }


"""

generation_config = {
    "max_output_tokens": 1000,
    "temperature": 0,
    "top_p": 0.9,
}
safety_settings = {
    generative_models.HarmCategory.HARM_CATEGORY_HATE_SPEECH: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
    generative_models.HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
    generative_models.HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
    generative_models.HarmCategory.HARM_CATEGORY_HARASSMENT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
}

def generate(a):
  vertexai.init(project="bigquerycourse-onetwothree", location="us-central1")
  model = GenerativeModel("gemini-1.5-pro-preview-0514")
  pdf_file_uri = a
  pdf_file = Part.from_uri(pdf_file_uri, mime_type="application/pdf")
  responses = model.generate_content(
      [prompt, pdf_file],
      generation_config=generation_config,
      safety_settings=safety_settings,
      stream=False
  )
  json_string = responses.text
  cleaned_string = json_string[json_string.find("{"):json_string.rfind("}") + 1]
  print(cleaned_string)
  return json.loads(cleaned_string)



In [4]:
# Loop through Contracts
df = pd.DataFrame()
for file_path in pdf_uris:
    dict1 = generate(file_path)  # Output from Gemini Python Dictionary
    df = pd.concat([df, pd.DataFrame.from_dict([dict1])], ignore_index=True) # Append to Dataframe


{
  "Supplier": "Dunn Transportation",
  "Purchaser": "The City of Tempe",
  "Description_of_Services": "Bus shuttling services to transport persons from the parking lot located on Hardy and Rio Salado to the Tempe Center of the Arts (TCA).",
  "Length_of_Service": "One year",
  "Break_Clause_Included": "Yes",
  "Description_of_Break_Clause": "At any time during the life of the agreement, the City may issue a termination for convenience, without default, by providing a written 30-day notice of termination to the other party.",
  "Value_of_Contract": "Not specified",
  "Currency": "USD",
  "Signature_Date": "July 29, 2019",
  "Contract_Expiry_Date": "August 1, 2020",
  "Payment Terms": "Net 30 days",
  "Total_Contract_Lifetime_Value": "Not specified",
  "Supplier_Country": "United States",
  "Purchaser_Country": "United States",
  "Work_Country": "United States",
  "Contract_Duration": "1 year",
  "Number_of_Pages": "5",
  "Payment Terms": "Net 30 days"
}
{
  "Supplier": "Life Quest Tra

In [5]:
df

,Supplier,Purchaser,Description_of_Services,Length_of_Service,Break_Clause_Included,Description_of_Break_Clause,Value_of_Contract,Currency,Signature_Date,Contract_Expiry_Date,Payment Terms,Total_Contract_Lifetime_Value,Supplier_Country,Purchaser_Country,Work_Country,Contract_Duration,Number_of_Pages
0,Dunn Transportation,The City of Tempe,Bus shuttling services to transport persons fr...,One year,Yes,"At any time during the life of the agreement, ...",Not specified,USD,"July 29, 2019","August 1, 2020",Net 30 days,Not specified,United States,United States,United States,1 year,5
1,"Life Quest Training & Consulting, LLC.",City of Tempe,Evaluation services for Tempe's Building Emplo...,One year,Yes,The City may terminate the agreement in whole ...,"$20,000",USD,"November 20, 2018","September 30, 2019",Net 30,"$20,000",United States,United States,United States,1 year,7


## Output to Big Query

In [ ]:
from google.cloud import bigquery

# Construct a BigQuery client object.
client = bigquery.Client(project="bigquerycourse-onetwothree")

# Indentify BQ Table
table_id = "glisten-414816.contracts.contracts"

# Load Results to BQ table
job = client.load_table_from_dataframe(
    df, table_id
)
job.result()


LoadJob<project=glisten-414816, location=US, id=94f455d2-615b-4842-9cc1-3e581e7e4b6f>

## Output to CSV

In [ ]:
# Specify the filename and path (optional, defaults to current directory)
filename = "output.csv"

# Save the DataFrame to a CSV file
df.to_csv(filename)